
## Классификация изображений из датасета [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html)


## Задание:

Реализовать алгоримтм позволяющий классифицировать изображения из датасета CIFAR-10 и оценить качество его работы.

Требования:

1. Можете придумать свой алгоритм, загрузить готовую модель или использовать собственную архитектуру.

2. Не используйте предобученные модели.

3. Выберите способ оценки качества предсказаний модели. Обоснуйте его.

4. Проведите обучение. Продемонстрируйте умение использовать соответствующие инструменты.

5. Оцените полученный результат.

*Не используйте инструменты принцип работы которых вам непонятен.

In [ ]:
%pip install torchvision
%pip install scikit-learn
%pip install numpy
%pip install pillow
%pip install scikit-image

### Данные

In [19]:
from torchvision import models, datasets
from torchvision.transforms import ToTensor

# Download a CIFAR10 dataset
dataset = datasets.CIFAR10("content", train=True, download=True, transform=ToTensor())

Files already downloaded and verified


In [30]:
class_names = dataset.classes

for i, name in enumerate(class_names):
    print(f"{i}: {name}")

0: airplane
1: automobile
2: bird
3: cat
4: deer
5: dog
6: frog
7: horse
8: ship
9: truck


In [31]:
import numpy as np
from skimage.transform import resize

labels = np.array([label for image, label in dataset])
data = np.array([np.array(image).flatten() for image, label in dataset])

# Сжатие изображения до 24x24 пикселей (для ускорения обучения)
data = np.array([resize(image.reshape(32, 32, 3), (24, 24, 3)).flatten() for image in data])

print(data)
print(labels)

[[0.22723313 0.20370372 0.26111114 ... 0.46013075 0.2697168  0.26830065]
 [0.5633987  0.50228757 0.45653597 ... 0.5002179  0.5074074  0.5205883 ]
 [0.99891067 0.99346405 0.99346405 ... 0.31383443 0.31753814 0.32712418]
 ...
 [0.20544665 0.19498911 0.1969499  ... 0.30446625 0.2724401  0.21252725]
 [0.7639434  0.7523965  0.7490196  ... 0.6149238  0.6518519  0.64738566]
 [0.858061   0.8898693  0.8955338  ... 0.69978213 0.6877996  0.68867105]]
[6 9 9 ... 9 1 1]


In [32]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Разделение на тренировочные и тестовые данные (чтобы избежать переобучения)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=45)

# Для SVM используем 50% данных для ускорения обучения (иначе долго будет обрабатываться, т.к. большой объём датасета)
subset_size = int(0.5 * len(dataset))
subset_indices = np.random.choice(len(dataset), subset_size, replace=False)
subset_data = data[subset_indices]
subset_labels = labels[subset_indices]

X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(subset_data, subset_labels, test_size=0.2, random_state=67)

# SVM чувствительна к масштабу данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_scaled)
X_test_scaled = scaler.transform(X_test_scaled)

### Модель

##### С этими моделями выполняла семинарские задачи по мат методам на 3-м курсе

In [33]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100)

In [34]:
from sklearn.svm import SVC

svm = SVC(kernel='rbf', C=10, gamma='scale')

# радиальное базисное ядро подходит для более сложных границ разделения, по идее для изображений как раз подойдет
# scale gamma рекомендуется для больших данных
# через увеличение C уменьшаем регуляризацию -- появляется больше границ между классами

### Обучение

In [35]:
rfc.fit(X_train, y_train)

RandomForestClassifier()

In [36]:
svm.fit(X_train_scaled, y_train_scaled)

SVC(C=10)

### Оценка результата

#### Выбраны метрики качества:

1. Accuracy, доля правильно классифицированных изображений, простой и наглядный способ оценки

2. Precision, Recall, F1-score (через classification_report): эти метрики полезны, т.к. учитывают баланс между ложноположительными и ложноотрицательными предсказаниями; показывают, как модель справляется с отдельными классами

3. Precision: как много предсказанных позитивных случаев являются истинно позитивными
4. Recall: сколько из всех истинных позитивных случаев было найдено
5. F1-score -- гармоническое среднее между precision и recall

6. Confusion Matrix: матрица показывает, какие классы модель путает между собой

In [37]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print('RandomForestClassifier metrics:\n')

y_pred_rfc = rfc.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred_rfc):.2f}\n")
print(classification_report(y_test, y_pred_rfc))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_rfc))

RandomForestClassifier metrics:

Accuracy: 0.46

              precision    recall  f1-score   support

           0       0.55      0.57      0.56      1017
           1       0.50      0.53      0.51      1026
           2       0.37      0.33      0.34       991
           3       0.35      0.30      0.32       945
           4       0.39      0.36      0.38      1047
           5       0.41      0.38      0.39       993
           6       0.44      0.53      0.48       986
           7       0.52      0.46      0.49      1039
           8       0.57      0.60      0.58       960
           9       0.49      0.56      0.52       996

    accuracy                           0.46     10000
   macro avg       0.46      0.46      0.46     10000
weighted avg       0.46      0.46      0.46     10000


Confusion Matrix:
[[581  52  43  23  31  20  27  26 163  51]
 [ 35 546  20  39  25  25  35  32  69 200]
 [ 93  46 323  66 136  73 116  74  34  30]
 [ 34  42  73 284  84 178 110  53  25  62]
 

In [38]:
print('SVM metrics:\n')

y_pred_svm = svm.predict(X_test_scaled)

print(f"Accuracy: {accuracy_score(y_test_scaled, y_pred_svm):.2f}\n")
print(classification_report(y_test_scaled, y_pred_svm))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test_scaled, y_pred_svm))

SVM metrics:

Accuracy: 0.53

              precision    recall  f1-score   support

           0       0.53      0.61      0.57       503
           1       0.63      0.63      0.63       511
           2       0.42      0.44      0.43       504
           3       0.36      0.38      0.37       502
           4       0.45      0.45      0.45       483
           5       0.43      0.40      0.41       499
           6       0.57      0.58      0.57       499
           7       0.63      0.55      0.59       498
           8       0.68      0.64      0.66       483
           9       0.60      0.58      0.59       518

    accuracy                           0.53      5000
   macro avg       0.53      0.53      0.53      5000
weighted avg       0.53      0.53      0.53      5000


Confusion Matrix:
[[308  21  30  17  21  12  10  15  41  28]
 [ 27 324   9  12   8  12  12  13  22  72]
 [ 43   7 221  39  63  41  50  20  10  10]
 [ 20  13  49 192  20  98  53  22  15  20]
 [ 37   6  78  34 21

## Вывод

#### Получен сомнительный результат точности моделей: 0.46 и 0.53, выбранные модели -- далеко не лучший выбор для классификации изображений. Читала, что свёрточные нейронные сети хороши в этой области, но с ними я не работала

#### Лучше всего распознаются классы (более высокие показатели метрик precision, recall и f1-score): 0 (самолет), 1 (автомобиль), 6 (лягушка), 7 (лошадь), 8 (корабль) и 9 (грузовик) — наверное, из-за чётких визуальных границ эти объекты более узнаваемы (особенно техника)

#### С классами 2 (птица), 3 (кошка), 4 (олень), 5 (собака) возникают сложности распознавания: причинами могут являться разнообразие видов (что усложняет классификацию в плане определения общего) -- для 2, 3 и 5; в случае 4 причиной может быть потеря рогов при сжатии и схожесть с лошадью

#### По матрице ошибок можно предположить, что класс автомобиля местами путается с классом грузовика, а класс самолета с классом корабля, что по идее можно объяснить схожим внешним видом конструкций

#### Cжатие изображений до меньшего разрешения (24x24 пикселя) скорее всего повлияло на ухудшение качества распознавания, особенно для менее чётких объектов (2, 3, 5), как и урезание выборки для SVM


Будьте готовы ответить на любые вопросы по коду, процессу обучения, моделям и метрикам.